# Imports

In [14]:
import tensorflow as tf
import os

## limit memory growth


In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)